In [18]:
import rasterio
from glob import glob
import numpy as np
import pandas as pd
import pickle
import os


In [19]:
##### START OPTIONS #####
root_path = "/data/"
stats_folder = root_path + "band_stats/"
chip_dir_filt = root_path + 'chips_filtered/'
chip_dir_binary_filt = root_path + 'chips_binary_filtered/'
chip_dir_multi_filt = root_path + 'chips_multi_filtered/'

#####  END OPTIONS  #####

In [20]:
dirs_to_make = [stats_folder]
for folder in dirs_to_make:
    try:
        os.makedirs(folder)
    except FileExistsError:
        # directory already exists
        print('pass')
        pass

pass


read in values for all HLS bands, one band at a time. save values on a per band basis

In [21]:
chips = glob(chip_dir_filt + "*_merged.tif")
print(len(chips))
means = []
stds = []

for band in range(1, 13):
    print(band)
    vals_all = np.ndarray([0])
    for k in range(len(chips)):
     #   print(k)
        file = chips[k]
        with rasterio.open(file) as src:
            vals = src.read(band).flatten()
            vals_all = np.concatenate([vals_all, vals])
    mean = vals_all.mean()
    std = vals_all.std()
    print(mean)
    print(std)
    means.append(mean)
    stds.append(std)
    
    with open(stats_folder + 'band_values_' + str(band) + '.list', 'wb') as file:
        pickle.dump(vals_all, file)
        



        
    #out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)


4238
1
447.6512258304285
171.39312946409086
2
706.6825287823107
230.2506792460155
3
879.0960315583436
350.9528178396797
4
2114.2992681052924
637.1488432543365
5
349.5627358616029
159.35557912214958
6
641.0681161537439
196.95454815196112
7
615.8145881146783
321.91264645088404
8
2576.2793531140373
627.8817516633674
9
336.65180812243125
161.51306914484033
10
612.3442885761327
207.00462718449378
11
611.4553372788473
352.6264694215877
12
2588.473145998478
569.9283100640438


In [22]:
print(means)
print(stds)

[447.6512258304285, 706.6825287823107, 879.0960315583436, 2114.2992681052924, 349.5627358616029, 641.0681161537439, 615.8145881146783, 2576.2793531140373, 336.65180812243125, 612.3442885761327, 611.4553372788473, 2588.473145998478]
[171.39312946409086, 230.2506792460155, 350.9528178396797, 637.1488432543365, 159.35557912214958, 196.95454815196112, 321.91264645088404, 627.8817516633674, 161.51306914484033, 207.00462718449378, 352.6264694215877, 569.9283100640438]


save band means and sds to csv

In [23]:
band_names = list(range(1, 13))
summary_df = pd.DataFrame({"band": band_names  ,  "mean": means, "sd": stds})
summary_df
summary_df.to_csv(stats_folder + "means_sds.csv")

consolidate means, sds for dates

In [24]:
means = []
stds = []

for item in [[1,5,9], [2,6,10], [3,7,11], [4,8,12]]:
    print(item)
    band_vals = []
    for band in item:
        print(band)
        with open(stats_folder + 'band_values_' + str(band) + '.list', 'rb') as f:
            x = pickle.load(f)
            print(x.mean())
            print(x.std())
            band_vals.append(x)
            del x

    band_vals = np.concatenate((band_vals[0], band_vals[1], band_vals[2]))
    mean = band_vals.mean()
    std = band_vals.std()
    means.append(mean)
    stds.append(std)
    del band_vals
        
            

[1, 5, 9]
1
447.6512258304285
171.39312946409086
5
349.5627358616029
159.35557912214958
9
336.65180812243125
161.51306914484033
[2, 6, 10]
2
706.6825287823107
230.2506792460155
6
641.0681161537439
196.95454815196112
10
612.3442885761327
207.00462718449378
[3, 7, 11]
3
879.0960315583436
350.9528178396797
7
615.8145881146783
321.91264645088404
11
611.4553372788473
352.6264694215877
[4, 8, 12]
4
2114.2992681052924
637.1488432543365
8
2576.2793531140373
627.8817516633674
12
2588.473145998478
569.9283100640438


In [25]:
print(means)
print(stds)

[377.9552566048209, 653.3649778373957, 702.1219856506231, 2426.3505890726024]
[171.489470178614, 215.51029055112002, 364.2939656160049, 650.9354309538619]


In [26]:
band_names = list(range(1, 5))
summary_df = pd.DataFrame({"band": band_names  ,  "mean": means, "sd": stds})
summary_df
summary_df.to_csv(stats_folder + "means_sds_consolidated_dates.csv")

calculate counts for all CDL classes

In [27]:
chips = glob(chip_dir_filt +  "*.mask.tif")
print(len(chips))
means = []
stds = []

for band in range(1, 2):
    print(band)
    vals_all = np.ndarray([0])
    for k in range(len(chips)):
     #   print(k)
        file = chips[k]
        with rasterio.open(file) as src:
            vals = src.read(band).flatten()
            if(len(vals) != 50176):
                print(k)
                print(file)
                print(len(vals))
            vals_all = np.concatenate([vals_all, vals])
    mean = vals_all.mean()
    std = vals_all.std()
    print(mean)
    print(std)
    means.append(mean)
    stds.append(std)
    
    with open(stats_folder + 'CDL_values.list', 'wb') as file:
        pickle.dump(vals_all, file)
        


4238
1
140.66307033879724
48.328550364210074


export CDL counts 

In [28]:
len(vals_all)
unique, counts = np.unique(vals_all, return_counts=True)
total_count = counts.sum()
pcts = (100.0*counts)/total_count
freq_table =  pd.DataFrame({"CDL_val": unique  ,  "count": counts, "pct": pcts})

In [29]:
freq_table.to_csv(stats_folder + "cdl_freq.csv")

In [30]:
counts.sum()

212645888